##### A work in progress: This notebook is a work in progress, so there might be a few bugs here and there, and parts of it are not (yet) pretty, but i think some salient concepts are represented, and overall, it's getting there...

### The "Adds up to 100" puzzle:
- The question is more or les stated, "Given some vector of "numbers" $X$ of positive numbers, which might be quite large, how many pairs of numbers add up to 100?"
- First, answer the question with a question: What kind of numbers? Integers? Over what range -- are all of the numbers $x_j < 100$?
- Here, we will provide reasonably general solutions for both the continuum and integer cases.
- Note that fundamentally, this is an $ON^2$ problem, but we want to reduce that as much as possible.e 

#### Summary findings:
- We can use sorting based strategies to significantly reduce the order of the algorithm for continuum sequences, though the performance of such an algorithm might be difficult to anticipate
- For integers, we can reduce the problem to approximately $ON$, or $c \to ON$ in the limit of very large $N$. This reduction arises from the simple fact that, no matter how large $N$ is, there are only 101 distinct values of interest, all positive integers $x \le 100$, including $0$. Therefore, we can index the set like, $I = \{ x:n_x, ... \}$ where $0 \le x \le 100$ are the values of $x$ and $n_x$ are the number of that value in the set.
- We can now look for the number of pairs in this reduced set, and in fact, because the valuse are integer type, we can again use our index. For each value $x_j$, we can compute its compliment, $x^c_j = 100 - x_j$; we can then look up $x^c_j$ in our index, and the

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline



In [2]:
import matplotlib
import pylab as plt
import numpy
import scipy
import random


### Make a sequence of numbers

In [3]:
N = 1000
R = random.Random()

x_min = 0.
x_max = 100.

# search range:
x_lo = 50.
x_hi = 60.

X = [R.random()*(x_min + (x_max - x_min)) for _ in range(N)]


### Continuum case:
- First, we might assume based on how the question was phrased that we can ignore this case. Continuum values never really equal oneanother, so we'll modify the question to say something like, find the pairs of numbers whose sum is between $x_{min} < x_1 + x_2 < x_{max}$.
- We don't have too much to work with; this problem is still fundamentally (as much as) $ON^2$, but:
    - We can use sorting to reduce the scope a bit
    - Obviuosly, we can exclude any numbers $x>x_{max}$
    - ... etc.
- So this gets us approximately $O(N \cdot \log(N) + \alpha N^2)$, where $\alpha$ is more or less (exactly?) the square of the fraction of values $x < x_{max}$.
- So we observe significant reductions in the complexity, even for $x_{hi} \approx x_{max}$. The next step might be to figure out a witty indexing or modulus-skipping system to optimize the search for small $x$.


In [4]:
# now, find all pairs where x1 < (x_j + x_k) < x2
#
# make a sorted copy of X:
X_prime = list(sorted(X))
pairs = []
nits = 0
#
# we should maybe (or maybe not) use itertools, but first, let's just block it out:
for j, x1 in enumerate(X_prime):
    # is this a valid value for x1? if we've already exceeded our maximum value, we can stop spinning...
    if x1>x_hi: break
    #
    for k, x2 in enumerate(X_prime):
        nits += 1
        if x2 + x1 > x_hi:
            break
        x = x1 + x2
        if x>=x_lo and x<=x_hi:
            pairs += [[j,k, x1, x2, x]]
        #
#
#       

print("Number of distinct pairs: {}".format(len(pairs)))
print('nits: {}/{} ({:.3f})'.format(nits, N*N, nits/(N*N)))

Number of distinct pairs: 56442
nits: 189100/1000000 (0.189)


#### Variable Types: The Integer trick

- So that's the sort of problem we would probably encounter in physical science -- continuum values.
- But in CS, or just for the sake of puzzles, let's consider the case where $X$ is restricted to positive ingegers, this problem becomes approximately $ON$ for large $N$. The number of integers that add to 100 is small (it's 100).
- We solve the problem in two steps. We index and filter the input:
    - Exclude all values $x>100$; count and index the number of each value $x \le 100$
    - Now, for each entry $x_j$, compute the colplement $x_k = 100 - x_j$, and add the total count, $n = n_j \cdot n_k$.
    - Note that we return the pairs, $x_j + x_k = 100$, and their counts. These are $2 \times$ degenerate, in that we get two redundant pairs, $(x_j, x_k) = (x_k, x_j)$, so if we want the total count, add these up and divide by two. 
    - ...or equivalently, add up half of them. So we only need to spin over $0 < x_1 < 50$.
- Note also that, we interpret this challenge with the simplified rule that an element can be use multiple times, in multiple pairs. If we can only use an entry once, the number of single-use pairs is $n=min(n_j, n_k)$.

In [5]:
N = 10000
R = random.Random()
#
x_min = 0.
x_max = 200.
#
X = [int(R.random()*(x_min + (x_max - x_min))) for _ in range(N)]
#
X_index = {x:0 for x in range(int(x_min), 101, 1) }
for x in X:
    if x>100: 
        continue
    #
    X_index[x] +=1
#
pairs = []
#
for x,n in X_index.items():
    # we might sort, or we can just skip x>x_0 values...
    if x>50: continue
    #if x>100: continue
    x_prime = int(100 - x)
    n_prime = X_index[x_prime]
    #
    pairs += [[(x, x_prime), (n, n_prime), n*n_prime]]
# ... but we will still count 50 twice.note that 50+50 should occur approximately half as often
# as other combinations, since it is the same one value repeated, not two values.
# generally, this might be considered an example of the kind of erro we might incur by trying to
# over optimize -- introducing complexities to the code that lend themselves to mistakes, but don't
# really add much benefit (how much do we really need the O(N/2) boost? at what cost in terms of possible
# errors?)
# now
pairs[50][-1]/=2
n_pairs = numpy.sum([rw[-1] for rw in pairs])
#
#
# a quick idot-check:
print('n[100] = {}, n[3]={}, n[5]={}'.format(X.count(100), X.count(3), X.count(5)))
print('total pairs: {}'.format(n_pairs))
#
print('pairs: ')
for p in pairs: print(p)


n[100] = 48, n[3]=56, n[5]=46
total pairs: 126579.0
pairs: 
[(0, 100), (41, 48), 1968]
[(1, 99), (54, 59), 3186]
[(2, 98), (50, 51), 2550]
[(3, 97), (56, 49), 2744]
[(4, 96), (52, 48), 2496]
[(5, 95), (46, 63), 2898]
[(6, 94), (46, 46), 2116]
[(7, 93), (54, 46), 2484]
[(8, 92), (43, 62), 2666]
[(9, 91), (45, 40), 1800]
[(10, 90), (47, 47), 2209]
[(11, 89), (50, 54), 2700]
[(12, 88), (50, 38), 1900]
[(13, 87), (64, 53), 3392]
[(14, 86), (49, 39), 1911]
[(15, 85), (45, 62), 2790]
[(16, 84), (57, 51), 2907]
[(17, 83), (56, 53), 2968]
[(18, 82), (59, 53), 3127]
[(19, 81), (63, 51), 3213]
[(20, 80), (65, 51), 3315]
[(21, 79), (53, 51), 2703]
[(22, 78), (35, 51), 1785]
[(23, 77), (48, 40), 1920]
[(24, 76), (48, 61), 2928]
[(25, 75), (52, 66), 3432]
[(26, 74), (45, 51), 2295]
[(27, 73), (44, 46), 2024]
[(28, 72), (52, 53), 2756]
[(29, 71), (50, 53), 2650]
[(30, 70), (48, 54), 2592]
[(31, 69), (40, 41), 1640]
[(32, 68), (57, 44), 2508]
[(33, 67), (45, 49), 2205]
[(34, 66), (41, 50), 2050]
[(35

In [6]:
# now, code it up into a class with some generalizations:

class Pair_Sum_Counter(list):
    def __init__(self, X, x_target=100, enforce_integers=True):
        # X: input vector. we are going to assume (and enforce) ingegers.
        # first, verify that we have integers (though we permit skipping this step, for large
        self.x_target = x_target
        self.enforce_integers = enforce_integers
        # data sets and confident operators):
        if enforce_integers:
            # check for integers, then cast? or just cast? since we don't know what we're going to get,
            # we're guessing at which approach optimizes, so just keep it simple.
            # we might also block out the loop so we can raise a warning for non-integer types,
            # but that will compromize performance in Python.
            X = [int(x) for x in X]
        #
        # now, index X:
        # to index all of X (number of each value):
        #self.update({x:0 for x in set(X)})
        #
        # but we really only want:
        X_index = {k:0 for k in range(0, x_target+1, 1)}
        #
        # now count:
        for x in X:
            if x>x_target: 
                continue
            #
            X_index[x] +=1
        #
        pairs = self
        #
        # for simplicity, let's just count all the pairs, recognize that we are double-counting the
        # (arguably) degerate cases (a,b) == (b,a). we can optimize by skipping the second half of the primary
        # loop, but then we have to handle odd/even cases separately (aka, for 50 + 50 = 100), and the main
        # objective of this algorithm is to reduce an ON^2 problem to ~O(x_target), so a factor of 2 here
        # is usually a small problem.
        for x,n in X_index.items():
            # we might sort, or we can just skip x>x_0 values...
            if x>x_target: continue
            x_prime = int(x_target - x)
            n_prime = X_index[x_prime]
            #
            pairs += [[(x, x_prime), (n, n_prime), n*n_prime]]
        #
        # this should always be even (sum([])/2 is integer)
        self.n_pairs = numpy.sum([rw[-1] for rw in pairs])/2.

In [11]:
# first, check vs. previous code:     
PSC = Pair_Sum_Counter(X=X, x_target=100)
print(PSC.n_pairs)

print('***')
print(pairs[0:5])
print('*****\n*****')
print(PSC[0:5])
#
# what about some other x_target?
PCS2 = Pair_Sum_Counter(X=X, x_target=42)
print(PCS2[0:5])

126579.0
***
[[(0, 100), (41, 48), 1968], [(1, 99), (54, 59), 3186], [(2, 98), (50, 51), 2550], [(3, 97), (56, 49), 2744], [(4, 96), (52, 48), 2496]]
*****
*****
[[(0, 100), (41, 48), 1968], [(1, 99), (54, 59), 3186], [(2, 98), (50, 51), 2550], [(3, 97), (56, 49), 2744], [(4, 96), (52, 48), 2496]]
[[(0, 42), (41, 54), 2214], [(1, 41), (54, 55), 2970], [(2, 40), (50, 40), 2000], [(3, 39), (56, 44), 2464], [(4, 38), (52, 58), 3016]]


#### Check results with brute-force method:

In [14]:
import itertools

class Brute_Force_Pair_Counter(list):
    # brute force pair-counter. assume integer inputs, or at least that we can use
    # a == b equivalence. for float values, we really need to write an almost_equal(a,b)
    # operator, but we'll save that for a real application...
    #
    # regarding list[] inheritance... or maybe dict. eventually populate the pairs... mabye use a nested
    # dict{} structure, {x_j:{x_k:n, ...}, }
    def __init__(self, X, target_value=100):
        n_pairs = 0
        for x1,x2 in itertools.product(X,X):
            if x1 + x2 == 100: n_pairs += 1
            #
        #
        self.n_total = n_pairs
    #
BF_pairs = Brute_Force_Pair_Counter(X,100)

print('total: {} ({}), ON2:{}/{}'.format(BF_pairs.n_total/2, n_pairs, len(X)**2, 2*numpy.log10(len(X))))
#print(n_pairs*2)


total: 126579.0 (126579.0), ON2:100000000/8.0


### Diagnostics and Unit Testing
- Test the integer counter class against the Brute_Force counter class for a few data sets, for a few x_target.
- Evaluate time-to-completion (TTC), n_steps (how did we do with complexity?), and of course to be sure we get the same count(s).

### Comments: 
- Note that we can hybridize these techniques.
- If we have inherently continuum value data, can we approximate (round or truncate) those values, so that we can index them?
    - $x = 5.1$, $dx = .1$, $j=51$
    - $j_k = int \left( \frac{x_k}{dx} \right)$ 
- It gets a little bit trickeier when we then evaluate a range of equalities, using our indexed system, but it's not too bad; we simply replace our $x_c = 100 - x$ with a framework where we evaluate all $(x_{lo}-x) < x_c < (x_{hi}-x)$. How exactly we do this may depend on the properties of the data.
